# Рабочий вариант курсовой работы

## Настройка ноутбука

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


MY_RANDOM = 427

## Определение функций

In [28]:
def create_lr_model(df, features):
    """ Создает модель линейной регрессии, делает прогноз на данных из df.
        Печатает оценки r2 и mse и возвращает кортеж из них
    """

    target = ['Price']

    X = pd.DataFrame(df[features])
    y = pd.DataFrame(df[target])

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=MY_RANDOM)

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_valid)

    r2 = r2_score(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)

    print(f'r2 = {r2}\nmse = {mse}')
    
    return (r2, mse)


## Данные

* **data_test** - test.csv
* **data_train** - train.csv
* **data_all** - совмещенные test и train (для анализа распределения, расчета производных значений, определения классов)
* **data** - данные train которые будем очищать
* **data_init** - данные для первичного предсказания

In [20]:
data_test = pd.read_csv('test.csv')
data_train = pd.read_csv('train.csv')
data_all = pd.concat([data_test, data_train.drop(['Price'], axis=1)])
data = data_train.copy()

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [22]:
# Поле Healthcare_1 уберем, т.к. в нем слишком много пропущенных значений (боле 47%)
data = data.drop(['Healthcare_1'], axis=1)

In [23]:
# Для первичного предсказания заполним пропущенные значения по полю LifeSquare средним значением
data_init = data.copy()
data_init.loc[:,'LifeSquare'] = data_init.loc[:,'LifeSquare'].fillna(data_train.LifeSquare.mean())

In [24]:
# Строки переведем в числа (0 и 1)
data_init.loc[data_init['Ecology_2'] == 'B', 'Ecology_2'] = 1
data_init.loc[data_init['Ecology_2'] == 'A', 'Ecology_2'] = 0

data_init.loc[data_init['Ecology_3'] == 'B', 'Ecology_3'] = 1
data_init.loc[data_init['Ecology_3'] == 'A', 'Ecology_3'] = 0

data_init.loc[data_init['Shops_2'] == 'B', 'Shops_2'] = 1
data_init.loc[data_init['Shops_2'] == 'A', 'Shops_2'] = 0

In [33]:
# Сделаем первичное предсказание и запомним начальные оценки точности

features = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
            'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
            'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
            'Shops_1', 'Shops_2']

print(f'Первичное предсказание:')

r2_init, mse_init = create_lr_model(data_init, features)

Первичное предсказание:
r2 = 0.367699524172983
mse = 5193081812.447669


In [36]:
# Пробуем убрать некоторые из признаков. Лучше всего модель показывает себя без признака LifeSquare

features_2 = [
                'DistrictId',     # 378 убрать
                'Rooms',          # 109 оставить, важный признак
                'Square',         # -1514 оставить, главный признак
#                 'LifeSquare',     # 450 убрать? почистить?
                'KitchenSquare',  # 372 убрать
                'Floor',          # 361 оставить
                'HouseFloor',     # 355 оставить
                'HouseYear',      # 372 убрать
                'Ecology_1',      # 360 оставить
                'Ecology_2',      # 375 убрать
                'Ecology_3',      # 372 убрать
                'Social_1',       # 391 убрать
                'Social_2',       # 374 убрать
                'Social_3',       # 372 убрать
                'Helthcare_2',    # 374 убрать
                'Shops_1',        # 380 убрать
                'Shops_2'         # 368 оставить
]

r2, mse = create_lr_model(data_init, features_2)

r2 = 0.4304206718699283
mse = 4677953224.358548


# Уберем аномалии

3. Заполнять пропущенные значения в поле LifeSquare будем средним значением LifeSquare для каждой группы квартир по количеству комнат на всех данных (data_all). Но перед этим необходимо почистить данные в полях Rooms и LifeSquare.

In [ ]:
columns = ['Rooms', 'LifeSquare']
data[columns].describe().T

Необходимо проверить:

+ квартиры с кол-ом комнат равном нулю, это либо ошибка, либо квартиры студии
+ квартиры с большим количеством комнат
+ аномально минимальные значения жилой площади
+ аномально макисмальные значения жилой площади

In [ ]:
data[data['Rooms'] == 0]

Для квартиры 1397 попробуем ориентироваться на цену и год постройки, т.к. сомнительно выглядят данные по полям Rooms, Square, LifeSquare, а так же Floor > HouseFloor

In [ ]:
data[data['HouseYear'] == 2016].groupby(by=['Rooms'])['Price'].agg(['mean', 'std', 'max', 'min'])

Видно, что квартира 1397 скорее всего 3-х комнатная.

Попробуем определить значения площадей.

In [ ]:
data[(data['HouseYear'] == 2016) & (data['Rooms'] == 3)][['Square', 'LifeSquare']].describe()

Значение Square для квартиры 1397 можно взять равным 83.576946